In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# 특정 워크플로우의 초기는 다음과 같이 생긴다
import uuid
from pathlib import Path
from typing import List, Dict
from datetime import datetime

from entity.process import GuardCondition, TaskSpec, TaskType, Layer, AgentNature, AgentRole, Process
from entity.validators import TokenValidator, SpecChainValidator
from entity.tokens import Token
from core.utils import load_resource_specs
import os
import logging
from core import logging_utils
from pathlib import Path
from core.tokenDB import TokenRepository
from core.utils import load_cfg

# 0. master config 불러오기
cfg = load_cfg(Path().parent.parent / "cfg" / "prod.yaml")


### 0. 노트북 작성목적

PROC_1 - 청킹 및 단순 적재 (Hard Process)
(금융연수원_소호_여신심사.md와 금융연수원_여신심사 및 관리.md)를 읽어들어, T1_LOAD, T1_SPLT, T1_TAG, T1_VECT의 태스크를 통해 "지식체계화"하는 것을 목표로 둔다. 즉, 최초로 형성되는 "비정형 지식 데이터베이스"의 자동화 프로세스 시도인 것이다.   
  
본 프로세스는 전부 비LLM함수로만 진행되는 단순 적재 Hard Process이다. 
지식체계화는 Proc_2, Proc_3, Proc_3에서 시도한다. 

내부 함수화, 즉 MCP_proxy는 스크립트 단위로 관리된다.  
현재 Hard Process를 불러오는 것도 없으니깐, 그것부터 정의를 한다. 

### 1. 최초 로드 및 DB 적재

### 2. ./Tools/tools:TokenizationPipeline 사용 예시

In [85]:
def load_markdown_files(data_path: str = "./data/input") -> dict[str, str]:
    """
    Update: 파일 경로가 들어오면 해당 파일만, 폴더가 들어오면 폴더 내 md 전체 로드
    """
    target_path = Path(data_path)
    
    # 1. 존재 여부 확인
    if not target_path.exists():
        print(f"[Error] 경로가 존재하지 않음: {data_path}")
        return {}

    # 2. Case A: 단일 파일인 경우
    if target_path.is_file():
        if target_path.suffix.lower() != '.md':
            print("[Warn] .md 파일이 아님")
            return {}
        return {target_path.stem: target_path.read_text(encoding="utf-8")}

    # 3. Case B: 디렉토리인 경우
    if target_path.is_dir():
        return {
            file.stem: file.read_text(encoding="utf-8")
            for file in target_path.glob("*.md")
        }

    return {}

class TokenizationPipeline:
    def __init__(self, db_repo: TokenRepository, chunk_size: int = 500, overlap: int = 50):
        self.repo = db_repo
        self.chunk_size = chunk_size
        self.overlap = overlap

    def _guess_topic(self, text: str) -> Dict[str, float]:
        """[Heuristic] 텍스트 키워드 기반 초기 Topic 추정 (LLM 전 단계)"""
        topics = {}

        # 향후 설정 필요
        if "매출" in text or "이익" in text: topics["TOPIC_FINANCE"] = 0.8
        if "담보" in text or "보증" in text: topics["TOPIC_COLLATERAL"] = 0.8
        if "소송" in text or "연체" in text: topics["TOPIC_RISK"] = 0.9
        
        if not topics: topics["TOPIC_GENERAL"] = 0.5
        return topics

    def _create_chunks(self, text: str) -> List[str]:
        """Fixed Size + Overlap 기반 청킹"""
        chunks = []
        start = 0
        text_len = len(text)
        
        while start < text_len:
            end = min(start + self.chunk_size, text_len)
            chunks.append(text[start:end])
            if end == text_len: break
            start += (self.chunk_size - self.overlap) # Overlap 이동
            
        return chunks

    def run(self, data_path: str):
        """Pipeline Execution: Load -> Split -> Tokenize -> Save"""
        # 1. Load
        raw_docs = load_markdown_files(data_path)
        if not raw_docs:
            print("[Info] 처리할 문서가 없습니다.")
            return

        total_tokens = 0
        
        for source_id, text in raw_docs.items():
            # 2. Split
            chunks = self._create_chunks(text)
            
            for idx, chunk_text in enumerate(chunks):
                # 3. Tokenize (Process Objectification)
                token = Token(
                    trace_id=f"TKN_{source_id}_{idx:04d}", # 순서 보장 ID
                    source_id=source_id,
                    history=["LOAD_SPLIT"],
                    content={
                        "text": chunk_text,
                        "chunk_index": idx,
                        "total_chunks": len(chunks),
                        "length": len(chunk_text),
                        "content_ref" : str(Path(data_path) / f"{source_id}.md")
                    },
                    topics=self._guess_topic(chunk_text)
                )
                
                # 4. Save
                self.repo.save(token)
                total_tokens += 1
                
        print(f"[Success] Pipeline Completed. Total Tokens Saved: {total_tokens}")

### 3. MD 파일 토큰 형태로 DB 적재

In [81]:
repo = TokenRepository(cfg['location']['token_db'])
pipeline = TokenizationPipeline(repo, chunk_size=500, overlap=50)
pipeline.run("./data/input/금융연수원_소호 여신심사.md")

[Success] Pipeline Completed. Total Tokens Saved: 461


In [86]:
pipeline.run("./data/input/금융연수원_여신심사 및 관리.md")

[Success] Pipeline Completed. Total Tokens Saved: 872


In [ ]:
repo.load("TKN_금융연수원_소호 여신심사_0311")